# MDI+: Example Usages

In [149]:
%reload_ext autoreload
%autoreload 2
import sys
sys.path.append("../")
sys.path.append("../../")

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, mean_absolute_error, accuracy_score, roc_auc_score, mean_squared_error

from imodels.importance import RandomForestPlusRegressor, RandomForestPlusClassifier, \
    RidgeRegressorPPM, LassoRegressorPPM, IdentityTransformer
from imodels.importance.rf_plus import _fast_r2_score

In [150]:
def neg_mae(y_true, y_pred, **kwargs):
    """
    Evaluates negative mean absolute error
    """
    return -mean_absolute_error(y_true, y_pred, **kwargs)


def neg_log_loss(y_true, y_pred, epsilon=1e-15):
    """
    Evaluates negative log-loss
    """
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)  # Clip predicted probabilities to avoid extreme values
    return np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

In [151]:
# helper variables
rf_regressor = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=331)
rf_classifier = RandomForestClassifier(n_estimators=100, min_samples_leaf=1, max_features="sqrt", random_state=331)

## 1. Regression Example

In [152]:
# generate data from linear model: y = x1 + x2 + N(0, 1)
n = 200
p = 10
s = 2
X = np.random.normal(size=(n, p))
# center and scale X
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)
beta = np.concatenate((np.ones(s), np.zeros(p-s)))
y = np.matmul(X, beta) + np.random.normal(size=n)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=12345)
# center and scale X_train and X_test
X_train = (X_train - np.mean(X_train, axis=0)) / np.std(X_train, axis=0)
X_test = (X_test - np.mean(X_test, axis=0)) / np.std(X_test, axis=0)

### 1.1 MDI+ with default settings for regression

In [153]:
# fit RF+
rf_plus_model = RandomForestPlusRegressor(rf_model=rf_regressor, include_raw=False)
rf_plus_model.fit(X_train, y_train)

In [154]:
# make predictions with RF+
preds = rf_plus_model.predict(X_test)
r2_score(y_test, preds)

0.4637529808576025

In [155]:
# get MDI+ scores (higher r^2 value = greater importance)
mdi_plus_scores = rf_plus_model.get_mdi_plus_scores(X_train, y_train, lfi = True)

len(coefs[j]): 15
len(loo_coefs[j]): 134
loo_coefs[j].shape: (134, 15)
Psi_k shape: (134, 14)
              0           1           2          3         4    5          6  \
0     58.395543  151.248882   46.092675  52.374254  1.380361  0.0   0.976062   
1      3.983275   89.340457  192.412961  52.390780  1.380796  0.0   0.976370   
2      3.983275   72.831824  192.412952  52.390778  1.380796  0.0   0.976370   
3      3.983261   14.617139   46.107148  52.390700  1.380794  0.0   0.976369   
4     58.433926  151.255518   46.124031  52.409883  1.381300  0.0   0.976726   
..          ...         ...         ...        ...       ...  ...        ...   
129  157.247661   61.006487   46.105328  52.388631  1.380739  0.0   0.976330   
130  105.425450   56.158084   46.107212  52.390772  1.380796  0.0  64.440427   
131  105.451999   56.174754   46.107205  52.390764  1.380796  0.0  64.440418   
132    3.983271   72.831847  192.412886  52.390760  1.380796  0.0   0.976370   
133   58.413543  151.25199

In [156]:
mdi_plus_scores["global"]

,var,importance
0,0,0.107294
1,1,0.194669
2,2,-0.000241
3,3,-0.004509
4,4,-0.004620
5,5,-0.003628
6,6,-0.002669
7,7,0.003634
8,8,-0.004237
9,9,-0.004441


In [157]:
pd.DataFrame(mdi_plus_scores["lfi"])

,0,1,2,3,4,5,6,7,8,9
0,0.288170,0.785813,0.074087,0.065894,0.082750,0.042553,0.073610,0.088608,0.073846,0.073691
1,0.251335,0.332541,0.194985,0.067020,0.055784,0.064182,0.087319,0.113234,0.090477,0.087500
2,0.251999,0.617243,0.169906,0.069663,0.058410,0.101240,0.069372,0.096354,0.087336,0.065535
3,0.522883,0.293222,0.076622,0.060611,0.053574,0.061087,0.081544,0.110186,0.114169,0.055041
4,0.548805,0.367264,0.054617,0.060482,0.078516,0.052544,0.075326,0.144235,0.070374,0.051857
...,...,...,...,...,...,...,...,...,...,...
129,0.602427,0.348315,0.068729,0.058754,0.075952,0.062783,0.076606,0.104671,0.054799,0.046964
130,0.589427,0.529139,0.105632,0.054717,0.049117,0.056373,0.063770,0.093585,0.054836,0.069003
131,0.552623,0.393476,0.067784,0.050956,0.073880,0.033496,0.082907,0.118455,0.071143,0.055156
132,0.257293,0.430597,0.144099,0.076313,0.069681,0.073098,0.072421,0.126522,0.081156,0.055457


#### 1.1.1. Local Feature Importances

In [158]:
# get global MDI+ scores using default scoring function (i.e., R^2 for regression so higher = greater importance)
# and local MDI+ scores using mean squared error (so lower = greater importance)
mdi_plus_scores = rf_plus_model.get_mdi_plus_scores(
    X_train, y_train, local_scoring_fns=mean_squared_error, version = "all", lfi=True
)

len(coefs[j]): 15
len(loo_coefs[j]): 134
loo_coefs[j].shape: (134, 15)
Psi_k shape: (134, 14)
              0           1           2          3         4    5          6  \
0     58.395543  151.248882   46.092675  52.374254  1.380361  0.0   0.976062   
1      3.983275   89.340457  192.412961  52.390780  1.380796  0.0   0.976370   
2      3.983275   72.831824  192.412952  52.390778  1.380796  0.0   0.976370   
3      3.983261   14.617139   46.107148  52.390700  1.380794  0.0   0.976369   
4     58.433926  151.255518   46.124031  52.409883  1.381300  0.0   0.976726   
..          ...         ...         ...        ...       ...  ...        ...   
129  157.247661   61.006487   46.105328  52.388631  1.380739  0.0   0.976330   
130  105.425450   56.158084   46.107212  52.390772  1.380796  0.0  64.440427   
131  105.451999   56.174754   46.107205  52.390764  1.380796  0.0  64.440418   
132    3.983271   72.831847  192.412886  52.390760  1.380796  0.0   0.976370   
133   58.413543  151.25199

In [159]:
mdi_plus_scores["global"].sort_values("importance", ascending=False)

,var,importance
1,1,0.194669
0,0,0.107294
7,7,0.003634
2,2,-0.000241
6,6,-0.002669
5,5,-0.003628
8,8,-0.004237
9,9,-0.004441
3,3,-0.004509
4,4,-0.004620


In [160]:
mdi_plus_scores["local"]

,0,1,2,3,4,5,6,7,8,9
0,3.157825,1.624339,3.316963,3.637423,3.821122,3.460957,3.532323,3.365652,3.558611,3.374752
1,1.579341,1.828877,1.270734,1.481250,1.596156,1.573449,1.494673,1.860829,1.446974,1.507208
2,4.236627,2.886514,4.138661,4.625807,4.517629,4.344145,4.661415,5.091227,4.564827,4.585125
3,2.214751,3.434221,3.499275,3.707769,3.560530,3.438775,3.466350,3.389409,3.952003,3.749907
4,1.402771,2.781199,2.714625,2.548685,2.598398,2.621838,2.648724,2.333774,2.615236,2.609043
...,...,...,...,...,...,...,...,...,...,...
129,2.480087,3.716359,4.090372,4.171840,4.587714,4.452727,4.495987,4.257148,4.254321,4.372481
130,0.826951,0.321770,0.073056,0.044354,0.040137,0.060029,0.037060,0.086606,0.063075,0.073406
131,0.401137,0.354582,0.028727,0.034580,0.035613,0.016483,0.041004,0.082889,0.045435,0.028366
132,1.561233,1.150931,1.744906,2.077754,1.876217,1.813400,1.925510,2.177293,1.899974,1.927208


In [161]:
pd.DataFrame(mdi_plus_scores["lfi"])

,0,1,2,3,4,5,6,7,8,9
0,0.288170,0.785813,0.074087,0.065894,0.082750,0.042553,0.073610,0.088608,0.073846,0.073691
1,0.251335,0.332541,0.194985,0.067020,0.055784,0.064182,0.087319,0.113234,0.090477,0.087500
2,0.251999,0.617243,0.169906,0.069663,0.058410,0.101240,0.069372,0.096354,0.087336,0.065535
3,0.522883,0.293222,0.076622,0.060611,0.053574,0.061087,0.081544,0.110186,0.114169,0.055041
4,0.548805,0.367264,0.054617,0.060482,0.078516,0.052544,0.075326,0.144235,0.070374,0.051857
...,...,...,...,...,...,...,...,...,...,...
129,0.602427,0.348315,0.068729,0.058754,0.075952,0.062783,0.076606,0.104671,0.054799,0.046964
130,0.589427,0.529139,0.105632,0.054717,0.049117,0.056373,0.063770,0.093585,0.054836,0.069003
131,0.552623,0.393476,0.067784,0.050956,0.073880,0.033496,0.082907,0.118455,0.071143,0.055156
132,0.257293,0.430597,0.144099,0.076313,0.069681,0.073098,0.072421,0.126522,0.081156,0.055457


In [162]:
# get global MDI+ scores using default scoring function (i.e., R^2 for regression so higher = greater importance)
# and local MDI+ scores using mean squared error (so lower = greater importance)
mdi_plus_scores_zach = rf_plus_model.get_mdi_plus_scores(
    X_train, y_train, local_scoring_fns=mean_squared_error, version = "sub"
)

len(coefs[j]): 15
len(loo_coefs[j]): 134
loo_coefs[j].shape: (134, 15)
Psi_k shape: (134, 14)
              0           1           2          3         4    5          6  \
0     58.395543  151.248882   46.092675  52.374254  1.380361  0.0   0.976062   
1      3.983275   89.340457  192.412961  52.390780  1.380796  0.0   0.976370   
2      3.983275   72.831824  192.412952  52.390778  1.380796  0.0   0.976370   
3      3.983261   14.617139   46.107148  52.390700  1.380794  0.0   0.976369   
4     58.433926  151.255518   46.124031  52.409883  1.381300  0.0   0.976726   
..          ...         ...         ...        ...       ...  ...        ...   
129  157.247661   61.006487   46.105328  52.388631  1.380739  0.0   0.976330   
130  105.425450   56.158084   46.107212  52.390772  1.380796  0.0  64.440427   
131  105.451999   56.174754   46.107205  52.390764  1.380796  0.0  64.440418   
132    3.983271   72.831847  192.412886  52.390760  1.380796  0.0   0.976370   
133   58.413543  151.25199

In [163]:
mdi_plus_scores_zach["global"].sort_values("importance", ascending=False)

,var,importance
1,1,0.198772
0,0,0.109884
7,7,0.003796
2,2,-0.000348
6,6,-0.002812
5,5,-0.003397
8,8,-0.004005
9,9,-0.004378
3,3,-0.004422
4,4,-0.004736


In [164]:
mdi_plus_scores_zach["local"]

,0,1,2,3,4,5,6,7,8,9
0,3.133415,1.611345,3.314210,3.639655,3.818102,3.453349,3.531678,3.366796,3.558983,3.374655
1,1.591783,1.814325,1.273527,1.477609,1.595416,1.577501,1.496993,1.860814,1.446073,1.506902
2,4.228420,2.855190,4.147549,4.620456,4.518759,4.349216,4.660343,5.094482,4.561200,4.586918
3,2.198160,3.429831,3.499911,3.712672,3.563483,3.434374,3.469310,3.387110,3.949988,3.748465
4,1.408235,2.747358,2.715189,2.542630,2.599331,2.624082,2.648841,2.335367,2.613942,2.607742
...,...,...,...,...,...,...,...,...,...,...
129,2.475473,3.705299,4.096179,4.176623,4.590265,4.447707,4.498862,4.255116,4.251951,4.374398
130,0.817446,0.319695,0.073651,0.043812,0.039690,0.060498,0.036817,0.086871,0.063072,0.073164
131,0.399374,0.346813,0.028214,0.035118,0.036812,0.016879,0.041128,0.082567,0.045904,0.028417
132,1.566299,1.142966,1.746327,2.073386,1.877182,1.813984,1.927477,2.178114,1.899662,1.927518


### 1.2 MDI+ with custom partial prediction model and evaluation metric(s)

In [165]:
# fit RF+ with custom partial prediction model
rf_plus_model = RandomForestPlusRegressor(rf_model=rf_regressor, prediction_model=LassoRegressorPPM())
rf_plus_model.fit(X_train, y_train)

In [166]:
# get MDI+ scores with custom evaluation metrics/scorers
mdi_plus_scores = rf_plus_model.get_mdi_plus_scores(X_train, y_train, scoring_fns={"r2_score": _fast_r2_score, "negative_mae": neg_mae})
mdi_plus_scores.sort_values("r2_score", ascending=False)

len(coefs[j]): 23
len(loo_coefs[j]): 134
loo_coefs[j].shape: (134, 23)
Psi_k shape: (134, 22)
              0           1          2          3         4    5          6  \
0     41.617180  129.951730  13.683285  15.548063  0.409780  0.0   0.289758   
1     22.517980    7.295298  57.137316  15.557520  0.410029  0.0   0.289935   
2      5.508735    7.295320  57.136501  15.557298  0.410024  0.0   0.289930   
3    163.816754    7.295505  13.688170  15.553614  0.409926  0.0   0.289862   
4    132.647211  129.960738  13.697799  15.564556  0.410215  0.0   0.290066   
..          ...         ...        ...        ...       ...  ...        ...   
129  236.440315    7.294091  13.685247  15.550293  0.409839  0.0   0.289800   
130  128.917021    7.295537  13.694551  15.560864  0.410118  0.0  19.139797   
131   59.971375    7.295322  13.691508  15.557407  0.410026  0.0  19.135544   
132   45.551942    7.295312  57.135925  15.557142  0.410019  0.0   0.289928   
133   78.132605  129.947861  13.69109

,var,r2_score,negative_mae
1,1,0.357189,-1.129205
0,0,0.228408,-1.166477
2,2,0.002794,-1.378339
7,7,0.000145,-1.380077
5,5,-0.002247,-1.383584
6,6,-0.002337,-1.384159
8,8,-0.002766,-1.384340
3,3,-0.002901,-1.383317
9,9,-0.003246,-1.384219
4,4,-0.003276,-1.385550


#### 1.2.1. Local Feature Importances

In [167]:
# get global and local MDI+ scores with custom evaluation metrics/scorers
mdi_plus_scores = rf_plus_model.get_mdi_plus_scores(
    X_train, y_train,
    scoring_fns={"r2_score": _fast_r2_score, "negative_mae": neg_mae},
    local_scoring_fns=True
)

len(coefs[j]): 23
len(loo_coefs[j]): 134
loo_coefs[j].shape: (134, 23)
Psi_k shape: (134, 22)
              0           1          2          3         4    5          6  \
0     41.617180  129.951730  13.683285  15.548063  0.409780  0.0   0.289758   
1     22.517980    7.295298  57.137316  15.557520  0.410029  0.0   0.289935   
2      5.508735    7.295320  57.136501  15.557298  0.410024  0.0   0.289930   
3    163.816754    7.295505  13.688170  15.553614  0.409926  0.0   0.289862   
4    132.647211  129.960738  13.697799  15.564556  0.410215  0.0   0.290066   
..          ...         ...        ...        ...       ...  ...        ...   
129  236.440315    7.294091  13.685247  15.550293  0.409839  0.0   0.289800   
130  128.917021    7.295537  13.694551  15.560864  0.410118  0.0  19.139797   
131   59.971375    7.295322  13.691508  15.557407  0.410026  0.0  19.135544   
132   45.551942    7.295312  57.135925  15.557142  0.410019  0.0   0.289928   
133   78.132605  129.947861  13.69109

/accounts/grad/zachrewolinski/research/imodels/notebooks/../imodels/importance/rf_plus.py:505: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - numerator / denominator


len(coefs[j]): 23
len(loo_coefs[j]): 134
loo_coefs[j].shape: (134, 23)
Psi_k shape: (134, 22)
              0          1          2           3          4           5    6  \
0     46.089585  56.898020   7.632468   31.537030  25.018145   35.743070  0.0   
1     36.237134  23.443619  48.703446   63.160706   3.655520  129.141596  0.0   
2     18.306239  87.175091  49.038984  182.281804  13.831373   53.205281  0.0   
3    191.762086   7.543469  42.945695   56.086199  18.693737    1.584741  0.0   
4    135.163780  13.536186  16.059930   37.740948  44.341321   33.192023  0.0   
..          ...        ...        ...         ...        ...         ...  ...   
129  220.361572  19.691060  21.025028  143.343910  35.645114   41.733280  0.0   
130  155.027563  37.590632  17.079725   17.995002  48.295722   20.039767  0.0   
131   82.424927  30.842779  28.422089   24.718740  36.816909   21.412697  0.0   
132   47.745144  24.868065  30.765955   97.825045  53.205373  114.386179  0.0   
133   77.762816

/accounts/grad/zachrewolinski/research/imodels/notebooks/../imodels/importance/rf_plus.py:505: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - numerator / denominator


len(coefs[j]): 20
len(loo_coefs[j]): 134
loo_coefs[j].shape: (134, 20)
Psi_k shape: (134, 19)
              0           1           2    3          4           5  \
0     24.941572   13.333212   22.923661  0.0  38.703780   54.491894   
1     22.099776   11.587808  188.303772  0.0   5.487629  174.062199   
2      5.679716   17.739089  189.272365  0.0  21.308167   81.506238   
3    158.481319   10.040701  138.953859  0.0  11.559764   17.769569   
4    131.747732  102.410306   48.213312  0.0  92.013924   25.626005   
..          ...         ...         ...  ...        ...         ...   
129  205.495972  102.993669   63.117059  0.0  37.915200   63.755601   
130  124.807713   12.945114   61.323970  0.0  57.584808   46.328553   
131   79.047133  102.560500   85.318374  0.0  57.045473   32.316566   
132   44.333999   11.716229  134.420463  0.0  82.521899  176.153633   
133   60.182114   12.035355   34.110505  0.0  33.746569   11.173288   

             6           7    8          9  
0    72.

...

/accounts/grad/zachrewolinski/research/imodels/notebooks/../imodels/importance/rf_plus.py:505: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - numerator / denominator


len(coefs[j]): 20
len(loo_coefs[j]): 134
loo_coefs[j].shape: (134, 20)
Psi_k shape: (134, 19)
              0          1          2          3           4    5          6  \
0     40.353179  35.453418   9.343167  14.562791   56.358566  0.0  67.539852   
1     24.013956  13.062520   9.342234  14.561336   17.189983  0.0  51.082064   
2      5.775368  20.100522   9.339375  14.556879   39.704630  0.0  29.250351   
3    175.537757   1.391401  39.848641  14.545765   17.724381  0.0  49.930588   
4    124.767611  11.958468   9.336157  14.551864  139.358796  0.0  48.689595   
..          ...        ...        ...        ...         ...  ...        ...   
129  213.213260   2.734696   1.044138  24.455788   60.346155  0.0   8.292854   
130  138.119757  14.622992   9.340113  14.558030   83.233779  0.0  17.210493   
131   64.179790  12.150496   9.338520  14.555547   82.463103  0.0  56.059771   
132   54.501406   3.312578   1.044107  24.455057  123.847853  0.0  88.727758   
133   72.092557   3.671077

/accounts/grad/zachrewolinski/research/imodels/notebooks/../imodels/importance/rf_plus.py:505: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - numerator / denominator


len(coefs[j]): 20
len(loo_coefs[j]): 134
loo_coefs[j].shape: (134, 20)
Psi_k shape: (134, 19)
              0           1          2          3    4    5          6  \
0     25.352571  209.213432   0.901820  28.253746  0.0  0.0  22.577721   
1     26.992526  126.376305  40.545095   1.501346  0.0  0.0   4.111518   
2      5.591628  417.462758   3.474656   5.779219  0.0  0.0   3.096318   
3    169.163804   56.084819   3.104596   1.500595  0.0  0.0   4.054061   
4    134.523640   82.565450   1.425491   1.500415  0.0  0.0   3.990527   
..          ...         ...        ...        ...  ...  ...        ...   
129  196.051042  109.764369   1.736132   1.500120  0.0  0.0   2.133132   
130  127.804917  198.318018   1.489632   5.782053  0.0  0.0   1.917222   
131   99.532977   90.105032   2.193436   1.499377  0.0  0.0   3.702696   
132   45.246598  142.080459   2.338441   5.777450  0.0  0.0   5.418350   
133   66.035295  147.229062  38.202671   1.499416  0.0  0.0   2.330925   

             7   

/accounts/grad/zachrewolinski/research/imodels/notebooks/../imodels/importance/rf_plus.py:505: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - numerator / denominator


In [168]:
mdi_plus_scores["global"].sort_values("negative_mae", ascending=False)

,var,r2_score,negative_mae
1,1,0.357189,-1.129205
0,0,0.228408,-1.166477
2,2,0.002794,-1.378339
7,7,0.000145,-1.380077
3,3,-0.002901,-1.383317
5,5,-0.002247,-1.383584
6,6,-0.002337,-1.384159
9,9,-0.003246,-1.384219
8,8,-0.002766,-1.384340
4,4,-0.003276,-1.385550


In [169]:
mdi_plus_scores["local"]["negative_mae"]

,0,1,2,3,4,5,6,7,8,9
0,-1.672240,-0.901932,-1.834223,-1.885467,-1.897952,-1.868807,-1.883571,-1.848458,-1.882404,-1.878433
1,-1.404889,-1.651099,-1.150579,-1.226096,-1.252553,-1.247656,-1.230491,-1.301085,-1.221529,-1.240043
2,-2.172731,-0.445342,-2.037112,-2.141883,-2.150749,-2.124933,-2.159365,-2.191218,-2.126251,-2.144530
3,-0.662701,-1.802728,-1.876077,-1.911754,-1.899086,-1.884252,-1.866766,-1.870486,-1.924000,-1.904197
4,-0.709614,-1.802203,-1.646154,-1.597449,-1.601214,-1.611596,-1.601819,-1.568395,-1.630514,-1.612673
...,...,...,...,...,...,...,...,...,...,...
129,-0.615583,-1.732214,-2.026389,-2.053088,-2.091595,-2.087298,-2.080458,-2.067927,-2.058390,-2.071889
130,-1.179031,-0.591014,-0.217110,-0.182290,-0.183888,-0.198019,-0.178852,-0.214412,-0.203158,-0.210654
131,-0.409822,-0.473103,-0.114433,-0.126105,-0.113503,-0.119039,-0.129892,-0.161101,-0.125912,-0.118738
132,-1.033883,-0.840684,-1.325674,-1.396986,-1.359155,-1.361470,-1.366667,-1.426504,-1.368467,-1.375927


### 1.3 MDI+ with custom transformer

The example below is equivalent to running RF+ with `include_raw=True`

In [170]:
# fit RF+ with custom transformer
rf_plus_model = RandomForestPlusRegressor(rf_model=rf_regressor, include_raw=False, add_transformers=[IdentityTransformer()])
rf_plus_model.fit(X_train, y_train)

In [171]:
# get MDI+ scores
mdi_plus_scores = rf_plus_model.get_mdi_plus_scores(X_train, y_train)
mdi_plus_scores.sort_values("importance", ascending=False)

len(coefs[j]): 23
len(loo_coefs[j]): 134
loo_coefs[j].shape: (134, 23)
Psi_k shape: (134, 22)
              0           1          2          3         4    5          6  \
0     43.666525  101.599610  17.954304  19.935193  3.235622  0.0   5.385608   
1     21.111327   24.415756  77.555543  19.930240  0.887891  0.0   4.152275   
2      5.629007   60.809393  77.603403  20.521584  2.006620  0.0   2.503918   
3    149.730782    9.524085  23.239187  20.665069  1.315703  0.0   4.055390   
4    126.520091   84.988475  19.225373  19.183250  5.355509  0.0   3.963709   
..          ...         ...        ...        ...       ...  ...        ...   
129  230.385645   29.848044  19.963768  23.254329  3.181580  0.0   0.925276   
130  135.754437   39.028472  19.372351  19.540982  4.564920  0.0  23.753321   
131   73.029478   25.858094  21.077036  19.740137  4.535735  0.0  26.679763   
132   42.076961   24.032190  74.864773  21.907981  6.336143  0.0   6.310514   
133   76.902530   93.624791  18.51941

,var,importance
1,1,0.355529
0,0,0.227631
7,7,0.001792
2,2,0.001506
8,8,-0.003163
3,3,-0.004336
4,4,-0.004418
5,5,-0.004432
6,6,-0.004650
9,9,-0.004685


### 1.4 Choosing the GLM and scoring metric via stability score

There are many choices of GLMs and scoring metrics that can be made within the MDI+ framework.

One way to select the GLM and scoring metric in MDI+ is by evaluating the stability of the feature importances/rankings for each choice of GLM/metric and taking the GLM/metric that is the most stable across different bootstrap samples of trees. For example, we can take the GLM and metric with the highest stability score, as measured by RBO below.

In [172]:
n_bootstraps = 25
prediction_models = {"ridge": RidgeRegressorPPM(), "lasso": LassoRegressorPPM()}
scoring_fns = {"r2": _fast_r2_score, "neg_mae": neg_mae}
stability_dict = {}
for model_name, prediction_model in prediction_models.items():
    # fit RF+
    rf_plus_model = RandomForestPlusRegressor(rf_model=rf_regressor, prediction_model=prediction_model)
    rf_plus_model.fit(X_train, y_train)
    # get MDI+ scores
    mdi_plus_scores = rf_plus_model.get_mdi_plus_scores(X_train, y_train, scoring_fns=scoring_fns)
    # get MDI+ stability scores
    mdi_plus_stability_scores = rf_plus_model.get_mdi_plus_stability_scores(B=n_bootstraps)
    stability_dict[model_name] = mdi_plus_stability_scores

len(coefs[j]): 23
len(loo_coefs[j]): 134
loo_coefs[j].shape: (134, 23)
Psi_k shape: (134, 22)
              0           1           2          3         4    5          6  \
0     49.525572  158.096384   25.496321  28.312723  2.634949  0.0   5.911971   
1     19.477670   33.756080  111.201881  28.298812  0.973696  0.0   4.582333   
2      5.804128   70.548620  111.290916  29.523583  1.765306  0.0   2.808652   
3    133.083084   12.678149   35.172399  29.823667  1.276606  0.0   4.480326   
4    122.707927  140.747485   27.820685  26.746888  4.136348  0.0   4.382205   
..          ...         ...         ...        ...       ...  ...        ...   
129  232.823157   45.937005   29.173390  35.188371  2.596456  0.0   1.108908   
130  140.234612   54.264389   28.092117  27.491073  3.577370  0.0  33.015152   
131   84.857640   40.519168   31.209614  27.904397  3.554491  0.0  36.165717   
132   37.993603   32.133800  106.280419  32.397107  4.828662  0.0   6.907448   
133   78.882768  149.76389

In [173]:
pd.concat(stability_dict, axis=0).reset_index().rename(columns={"level_0": "ppm"}).drop(columns=["level_1"]).sort_values("RBO", ascending=False)

,ppm,scorer,RBO,tauAP
1,ridge,neg_mae,0.942101,0.883994
3,lasso,neg_mae,0.905364,0.839801
0,ridge,r2,0.892978,0.760846
2,lasso,r2,0.885139,0.788898


### 1.5 Aggregating multiple MDI+ rankings in an ensemble

Instead of choosing a single GLM and metric to use in MDI+, it may be preferable in some cases to aggregate MDI+ feature importances/rankings across multiple choices of GLMs and metrics.

One naive method for doing this ensembling is to take the median rank across each choice of GLM and metric (as shown below). However, more creative ensembling schemes can also be explored.

In [174]:
prediction_models = {"ridge": RidgeRegressorPPM(), "lasso": LassoRegressorPPM()}
scoring_fns = {"r2": _fast_r2_score, "neg_mae": neg_mae}
mdi_plus_scores_dict = {}
for model_name, prediction_model in prediction_models.items():
    # fit RF+
    rf_plus_model = RandomForestPlusRegressor(rf_model=rf_regressor, prediction_model=prediction_model)
    rf_plus_model.fit(X_train, y_train)
    # get MDI+ scores
    mdi_plus_scores = rf_plus_model.get_mdi_plus_scores(X_train, y_train, scoring_fns=scoring_fns)
    for col in mdi_plus_scores.columns:
        if col != "var":
            mdi_plus_scores = mdi_plus_scores.rename(columns={col: model_name + "_" + col})
    mdi_plus_scores_dict[model_name] = mdi_plus_scores

len(coefs[j]): 23
len(loo_coefs[j]): 134
loo_coefs[j].shape: (134, 23)
Psi_k shape: (134, 22)
              0           1           2          3         4    5          6  \
0     49.525572  158.096384   25.496321  28.312723  2.634949  0.0   5.911971   
1     19.477670   33.756080  111.201881  28.298812  0.973696  0.0   4.582333   
2      5.804128   70.548620  111.290916  29.523583  1.765306  0.0   2.808652   
3    133.083084   12.678149   35.172399  29.823667  1.276606  0.0   4.480326   
4    122.707927  140.747485   27.820685  26.746888  4.136348  0.0   4.382205   
..          ...         ...         ...        ...       ...  ...        ...   
129  232.823157   45.937005   29.173390  35.188371  2.596456  0.0   1.108908   
130  140.234612   54.264389   28.092117  27.491073  3.577370  0.0  33.015152   
131   84.857640   40.519168   31.209614  27.904397  3.554491  0.0  36.165717   
132   37.993603   32.133800  106.280419  32.397107  4.828662  0.0   6.907448   
133   78.882768  149.76389

In [175]:
mdi_plus_scores_df = pd.concat([df.set_index('var') for df in mdi_plus_scores_dict.values()], axis=1)
mdi_plus_ranks_df = mdi_plus_scores_df.rank(ascending=False).median(axis=1)
mdi_plus_ranks_df = pd.DataFrame(mdi_plus_ranks_df, columns=["median_rank"]).reset_index()
mdi_plus_ranks_df.sort_values("median_rank")

,var,median_rank
1,1,1.0
0,0,2.0
2,2,3.5
7,7,3.5
3,3,6.5
5,5,6.5
8,8,6.5
6,6,8.0
9,9,8.5
4,4,10.0


## 2. Classification Example

In [176]:
# generate data from logistic model: logit(E[Y|X]) = x1 + x2
n = 200
p = 10
s = 2
X = np.random.normal(size=(n, p))
beta = np.concatenate((np.ones(s), np.zeros(p-s)))
probs = 1 / (1 + np.exp(-np.matmul(X, beta)))
y = (np.random.uniform(size=n) < probs) * 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=12345)

### 2.1 MDI+ with default classification settings

In [177]:
# fit RF+
rf_plus_model = RandomForestPlusClassifier(rf_model=rf_classifier)
rf_plus_model.fit(X_train, y_train)

In [178]:
# make predictions with RF+
preds = rf_plus_model.predict(X_test)
prob_preds = rf_plus_model.predict_proba(X_test)
accuracy_score(y_test, preds), roc_auc_score(y_test, prob_preds[:, 1])

(0.6515151515151515, 0.7096774193548387)

In [179]:
# get MDI+ scores (higher ngative log-loss value = greater importance)
mdi_plus_scores = rf_plus_model.get_mdi_plus_scores(X_train, y_train)
mdi_plus_scores.sort_values("importance", ascending=False)

len(coefs[j]): 30
len(loo_coefs[j]): 134
loo_coefs[j].shape: (134, 30)
Psi_k shape: (134, 29)
              0           1           2          3           4          5  \
0     17.846312   78.559400   24.677277  10.253279  115.806923  21.109926   
1     54.950977   78.464354   40.829524  13.292029   61.665309  41.744318   
2    173.840077   50.585026  105.257236   1.253522   49.879534  23.396327   
3     26.829727   56.634725   51.912559  24.584658   47.034094  35.152075   
4     22.594213   23.836403  200.498349  51.650277   45.319093   2.326973   
..          ...         ...         ...        ...         ...        ...   
129   40.585522   51.128827   65.199471  16.270200  149.831171  43.952608   
130  104.671039  308.257654   20.664183  30.508533   13.876428  10.094991   
131    6.931131   62.404937  388.151157  75.687228  108.857996  12.289161   
132   43.391939   66.703965   65.806312  53.342861  103.044135  20.236069   
133   78.464234   58.260552   41.924772  11.946413  110.725

,var,importance
1,1,-0.627285
0,0,-0.633117
6,6,-0.688551
3,3,-0.696212
4,4,-0.696713
2,2,-0.698729
5,5,-0.699714
7,7,-0.700604
9,9,-0.701797
8,8,-0.703259


#### 2.1.1. Local Feature Importances

In [180]:
# get MDI+ scores (higher ngative log-loss value = greater importance)
mdi_plus_scores = rf_plus_model.get_mdi_plus_scores(
    X_train, y_train, local_scoring_fns=neg_log_loss
)

len(coefs[j]): 30
len(loo_coefs[j]): 134
loo_coefs[j].shape: (134, 30)
Psi_k shape: (134, 29)
              0           1           2          3           4          5  \
0     17.846312   78.559400   24.677277  10.253279  115.806923  21.109926   
1     54.950977   78.464354   40.829524  13.292029   61.665309  41.744318   
2    173.840077   50.585026  105.257236   1.253522   49.879534  23.396327   
3     26.829727   56.634725   51.912559  24.584658   47.034094  35.152075   
4     22.594213   23.836403  200.498349  51.650277   45.319093   2.326973   
..          ...         ...         ...        ...         ...        ...   
129   40.585522   51.128827   65.199471  16.270200  149.831171  43.952608   
130  104.671039  308.257654   20.664183  30.508533   13.876428  10.094991   
131    6.931131   62.404937  388.151157  75.687228  108.857996  12.289161   
132   43.391939   66.703965   65.806312  53.342861  103.044135  20.236069   
133   78.464234   58.260552   41.924772  11.946413  110.725

In [181]:
mdi_plus_scores["global"].sort_values("importance", ascending=False)

,var,importance
1,1,-0.627285
0,0,-0.633117
6,6,-0.688551
3,3,-0.696212
4,4,-0.696713
2,2,-0.698729
5,5,-0.699714
7,7,-0.700604
9,9,-0.701797
8,8,-0.703259


In [182]:
mdi_plus_scores["local"]

,0,1,2,3,4,5,6,7,8,9
0,-0.685777,-0.397168,-0.780025,-0.761443,-0.776726,-0.830233,-0.747848,-0.821468,-0.768876,-0.762543
1,-0.947574,-0.240460,-0.575044,-0.648006,-0.528294,-0.595761,-0.723014,-0.597985,-0.623005,-0.553559
2,-0.198143,-0.792360,-0.852226,-0.753502,-0.827558,-0.811457,-0.745657,-0.790785,-0.983694,-0.790836
3,-0.710750,-0.599166,-0.779195,-0.810627,-0.795373,-0.828397,-0.715183,-0.800615,-0.968273,-0.783352
4,-0.619671,-0.562079,-0.846134,-0.771907,-0.749478,-0.760976,-0.906532,-0.792981,-0.698584,-1.039099
...,...,...,...,...,...,...,...,...,...,...
129,-0.367103,-0.625716,-0.599378,-0.613286,-0.611899,-0.611302,-0.618815,-0.586744,-0.571189,-0.607538
130,-0.211027,-0.198851,-0.586767,-0.580231,-0.615343,-0.569621,-0.663895,-0.592109,-0.584299,-0.657046
131,-0.849080,-1.117514,-0.798596,-0.947721,-0.669527,-0.774907,-0.917367,-0.778810,-1.016000,-0.775073
132,-1.228436,-0.511068,-0.801189,-0.805693,-0.690954,-0.801672,-1.270682,-0.798222,-0.795209,-0.774661


Note: `neg_log_loss()` was re-defined here since the default sklearn `log_loss()` metric requires vector inputs and thus cannot be directly used as a local scoring function.